In [ ]:
# Change these

locality = "nc-guilford"
verbose = True
clear_checkpoints = False
clear_model_results = False

# 1. Basic setup

In [ ]:
from init_notebooks import setup_environment
setup_environment()

In [ ]:
# import a bunch of stuff
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import os
import pandas as pd
from openavmkit.pipeline import (
    NotebookState, 
    set_locality,
    examine_df
)
from openavmkit.utilities.settings import (
    load_settings
)
from openavmkit.checkpoint import (
    from_checkpoint,
    read_checkpoint,
    write_checkpoint,
    delete_checkpoints
)
from openavmkit.benchmark import (
    run_models, 
    MultiModelResults
)

In [ ]:
if 'inited' not in globals():
    nbs: NotebookState = None
    inited = True
nbs = set_locality(nbs, locality)
settings = load_settings()

In [ ]:
if clear_checkpoints:
    delete_checkpoints("3-model")

# 2. Model

In [ ]:
# load the data
df = read_checkpoint("2-clean-03-out")

In [ ]:
results : MultiModelResults = run_models(
    df,
    load_settings(),
    use_saved_results=(not clear_model_results),
    verbose=verbose
)

# 3. Land Analysis

In [ ]:
def plot_histogram_df(df: pd.DataFrame, fields: str, xlabel: str = "", ylabel: str = "", title: str = "", bins = 500, x_lim=None, out_file: str = None):
    entries = []
    for field in fields:
        data = df[field]
        entries.append({
            "data": data,
            "label": field,
            "alpha": 0.25
        })
    plot_histogram_mult(entries, xlabel, ylabel, title, bins, x_lim, out_file)

def plot_histogram_mult(entries: list[dict],  xlabel:str = "", ylabel: str = "", title: str = "", bins=500, x_lim=None, out_file: str = None):
    plt.close('all')
    ylim_min = 0
    ylim_max = 0
    for entry in entries:
        data = entry["data"]
        if bins is not None:
            _bins = bins
        else:
            _bins = data.get("bins", None)
        label = entry["label"]
        alpha = entry.get("alpha", 0.25)
        data = data[~data.isna()]
        counts, _, _ = plt.hist(data, bins=_bins, label=label, alpha=alpha)
        _ylim_max = np.percentile(counts, 95)
        if(_ylim_max > ylim_max):
            ylim_max = _ylim_max
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    if x_lim is not None:
        plt.xlim(x_lim[0], x_lim[1])
    plt.ylim(ylim_min, ylim_max)
    if out_file is not None:
        plt.savefig(out_file)
    plt.show()

def highest_middle_quantile_count(series: pd.Series, min_value: float, max_value: float, num_quantiles:int):
    series = series[~series.isna()]
    series = series[series.ge(min_value) & series.le(max_value)]
    if num_quantiles < 3:
        raise ValueError("Number of quantiles must be at least 3")
    quantiles = pd.qcut(series, q=num_quantiles, duplicates="drop")
    quantile_counts = quantiles.value_counts()
    return quantile_counts.max()

In [ ]:
import pickle
%matplotlib inline

settings = load_settings()
instructions = settings.get("modeling", {}).get("instructions", {})
allocation = instructions.get("allocation", {})

outpath = "out/models"

results_map = {
    "main": {},
    "hedonic": {},
    "vacant": {}
}

for key in ["main", "hedonic", "vacant"]:
    if key == "main":
        models = instructions.get("main", {}).get("run", [])
        if "ensemble" not in models:
            models.append("ensemble")
    else:
        models = allocation.get(key, [])
    all_results = {}
    outpath = f"out/models/{key}"
    print(f"key = {key}")
    if len(models) > 0:
        for model in models:
            print(f"----> model = {model}")
            land_pred_univ = None
            pred_univ = None
            filepath = f"{outpath}/model_{model}.pickle"
            if os.path.exists(filepath):
                with open(filepath, "rb") as file:
                    results = pickle.load(file)
                    df = results.df_universe[["key"]].copy()
                    df.loc[:, "prediction"] = results.pred_univ
                    results_map[key][model] = df

df_all_alloc = results_map["main"]["ensemble"].copy()
all_alloc_names = []

bins = 400  

for key in ["hedonic", "vacant"]:
    df_alloc = results_map["main"]["ensemble"].copy()
    alloc_names = []
    entries = results_map[key]
    for model in entries:
        pred_main = results_map["main"].get(model)
        pred_land = results_map[key].get(model).rename(columns={"prediction": "prediction_land"})
        df = pred_main.merge(pred_land, on="key", how="left")
        alloc_name = f"{key}_{model}"
        df.loc[:, alloc_name] = df["prediction_land"] / df["prediction"]
        df.loc[df[alloc_name].gt(2.0), alloc_name] = None
        df.loc[df[alloc_name].lt(-0.5), alloc_name] = None
        df.loc[df[alloc_name].gt(1.0), alloc_name] = 1.0
        df.loc[df[alloc_name].lt(0.0), alloc_name] = 0.0
        df_alloc = df_alloc.merge(df[["key", alloc_name]], on="key", how="left")
        df_all_alloc = df_all_alloc.merge(df[["key", alloc_name]], on="key", how="left")
        alloc_names.append(alloc_name)
        all_alloc_names.append(alloc_name)

    df_alloc["allocation_ensemble"] = df_alloc[alloc_names].median(axis=1)
    
    # clip_height = float("-inf")
    # for alloc_name in alloc_names:
    #     _clip_height = highest_middle_quantile_count(df_all_alloc[alloc_name], 0.01, 1.0, bins)
    #     if _clip_height > clip_height:
    #         clip_height = _clip_height
    #     print(f"{alloc_name} _clip_height = {_clip_height} vs {clip_height}")
    
    plot_histogram_df(
        df=df_alloc, 
        fields=(alloc_names),
        xlabel="% of value attributable to land",
        ylabel="Number of parcels",
        title=f"Land allocation -- {key}",
        bins=bins,
        x_lim=(0.0, 1.0)
    )
    plot_histogram_df(
        df=df_alloc, 
        fields=["allocation_ensemble"],
        xlabel="% of value attributable to land",
        ylabel="Number of parcels",
        title=f"Land allocation -- {key}, ensemble",
        bins=bins,
        x_lim=(0.0, 1.0)
    )

plot_histogram_df(
    df=df_all_alloc, 
    fields=(all_alloc_names),
    xlabel="% of value attributable to land",
    ylabel="Number of parcels",
    title=f"Land allocation -- all",
    bins=bins,
    x_lim=(0.0, 1.0)
)
df_all_alloc["allocation_ensemble"] = df_all_alloc[all_alloc_names].median(axis=1)
plot_histogram_df(
    df=df_all_alloc,
    fields=["allocation_ensemble"],
    xlabel="% of value attributable to land",
    ylabel="Number of parcels",
    title=f"Land allocation -- all, ensemble",
    bins=bins,
    x_lim=(0.0, 1.0)
)